In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [ ]:
# data load-in

In [ ]:
##############hyper-parameters##############

tmp = math.exp(3)   # initial temperature
tmp_min = math.exp(-8)    # stop temperature
alpha = 0.98    # dropping rate of temperature
chrom_length = 8    # length of a chromosome
counter = 10000     # max iters allowed for generating a worse chrom

k = 0.002 # bigger k, more likely a worse chrom is accepted

columns = list(X_train.columns)
#############################################

In [ ]:
# get the initial chromosome
def geneEncoding():
    while True:
        temp = []
        has_1 = False   
        for j in range(chrom_length):
            rand = random.randint(0,1)
            if rand == 1:
                has_1 = True
            temp.append(rand)
        if has_1:   # can't be all zeros (must have at least one variable)
            return temp

In [ ]:
# accept or not (depending on delta_E)
def is_acceptable(delta_E,tmp):     
    if delta_E<=0:   # ΔE<=0，accept direcly
        print('accept directly')
        return True
    else:
        p=math.exp(-delta_E/(k*tmp))    # proba of accept
        r=random.random()
        if r<p:
            print(str(r)+"<"+str(p)+"，acceptable") 
            return True
        else:
            print(str(r)+">="+str(p)+"，reject"+"（"+str(counter)+"）")
            return False

In [ ]:
# generate a new chromosome from the old one
def getNewChrom(x):
    x_new = x.copy()
    mpoint = random.randint(0, chrom_length-1)  # randomly choose a mutation point
    if x[mpoint] == 1:
        x_new[mpoint] = 0
    else:
        x_new[mpoint] = 1
    return x_new

In [ ]:
def getFitness(x):
    X_sub = X_train

    has_1 = False
    for j in range(chrom_length):
        if x[j] == 0:
            col_name = columns[j]
            X_sub =X_sub.drop(columns = col_name)
        else:
            has_1 = True
        
    if has_1:
        clf = LogisticRegression(penalty='none') # logistic regression as the basic model
        fitness = cross_val_score(clf, X_sub, y_train, 
                                  cv=10, scoring='roc_auc').mean()  # 10-fold cv AUC as fitness
        return fitness
    else:
        fitness = 0     # 0 fitness for an all-zero chrom (i.e. at least one variable required)
        return fitness

In [ ]:
x_old = geneEncoding()
E_old = getFitness(x_old)
    
x_best = x_old
E_best = E_old

while tmp > tmp_min:

    x_new = getNewChrom(x_old)
    E_new = getFitness(x_new)
    delta_E = -(E_new - E_old)
    
    acc = is_acceptable(delta_E,tmp)
    
    if acc==True:
        x_old = x_new
        E_old = E_new
    
    if delta_E<=0:   # ΔE<=0，drop the temperature
        tmp = tmp * alpha
    else:
        counter -= 1

    if counter < 0:
        break
        
    if E_old >= E_best: # keep the best chrom along with its fitness
        E_best = E_old
        x_best = x_old
        
    print(tmp,x_old,E_old)
    print('best',x_best,E_best)
    print('\n')
    